In [1]:
import os
from cnnModel import get_model
from DataProvider import ChunkDoubleSourceSlider2
import NetFlowExt as nf
from Logger import log
import tensorflow as tf
import tensorflow.compat.v1 as tfc
from tensorflow import keras
from keras.layers import Input
import keras.backend as K
from tensorflow.python.keras.backend import get_session
from keras.utils import np_utils
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import argparse
from Arguments import *

2021-04-01 00:13:40,196 [INFO ]  Parameters: 
2021-04-01 00:13:40,198 [INFO ]  Machine name: Shalinis-MBP.attlocal.net
2021-04-01 00:13:42,458 [DEBUG]  Falling back to TensorFlow client; we recommended you install the Cloud TPU client directly with pip install cloud-tpu-client.
2021-04-01 00:13:43,402 [DEBUG]  $HOME=/Users/shalinimustala
2021-04-01 00:13:43,403 [DEBUG]  CONFIGDIR=/Users/shalinimustala/.matplotlib
2021-04-01 00:13:43,404 [DEBUG]  matplotlib data path: /Users/shalinimustala/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data
2021-04-01 00:13:43,409 [DEBUG]  loaded rc file /Users/shalinimustala/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/matplotlibrc
2021-04-01 00:13:43,411 [DEBUG]  matplotlib version 3.1.0
2021-04-01 00:13:43,412 [DEBUG]  interactive is False
2021-04-01 00:13:43,413 [DEBUG]  platform is darwin
2021-04-01 00:13:43,414 [DEBUG]  loaded modules: ['sys', 'builtins', '_frozen_importlib', '_imp', '_thread', '_warnings', '_weakref', 'zipimpor

2021-04-01 00:13:44,621 [DEBUG]  CACHEDIR=/Users/shalinimustala/.matplotlib
2021-04-01 00:13:44,625 [DEBUG]  Using fontManager instance from /Users/shalinimustala/.matplotlib/fontlist-v310.json
2021-04-01 00:13:44,828 [DEBUG]  Loaded backend module://ipykernel.pylab.backend_inline version unknown.
2021-04-01 00:13:44,831 [DEBUG]  Loaded backend module://ipykernel.pylab.backend_inline version unknown.


In [2]:
os.getcwd()

'/Users/shalinimustala/Documents/PPDS_coursework/DSCI560/Energy_disaggregation/Mypreprocessing_folder/transferNILM-master'

In [3]:
def remove_space(string):
    return string.replace(" ","")

In [4]:
def str2bool(v):
    if v.lower() in ('yes', 'true', 't', 'y', '1'):
        return True
    elif v.lower() in ('no', 'false', 'f', 'n', '0'):
        return False
    else:
        raise argparse.ArgumentTypeError('Boolean value expected.')

In [5]:
def get_arguments():
    parser = argparse.ArgumentParser(description='Train a neural network\
                                     for energy disaggregation - \
                                     network input = mains window; \
                                     network target = the states of \
                                     the target appliance.')
    parser.add_argument("-f", "--fff", help="a dummy argument to fool ipython", default="1")
    parser.add_argument('--appliance_name',
                        type=remove_space,
                        default='microwave',
                        help='the name of target appliance')
    parser.add_argument('--datadir',
                        type=str,
                        default= '/Users/shalinimustala/Documents/PPDS_coursework/DSCI560/Energy_disaggregation/Mypreprocessing_folder/trainingdata',
                        help='this is the directory of the training samples')
    parser.add_argument('--pretrainedmodel_dir',
                        type=str,
                        default='./pretrained_model',
                        help='this is the directory of the pre-trained models')
    parser.add_argument('--save_dir',
                        type=str,
                        default='./models',
                        help='this is the directory to save the trained models')
    parser.add_argument('--batchsize',
                        type=int,
                        default=1000,
                        help='The batch size of training examples')
    parser.add_argument('--n_epoch',
                        type=int,
                        default=5,
                        help='The number of epochs.')
    parser.add_argument('--save_model',
                        type=int,
                        default=-1,
                        help='Save the learnt model:\
                        0 -- not to save the learnt model parameters;\
                        n (n>0) -- to save the model params every n steps;\
                        -1 -- only save the learnt model params\
                        at the end of training.')
    parser.add_argument('--dense_layers',
                        type=int,
                        default=1,
                        help=':\
                                1 -- One dense layers (default Seq2point);\
                                2 -- Two dense layers;\
                                3 -- Three dense layers.')
    parser.add_argument("--transfer_model", type=str2bool,
                        default=False,
                        help="True: using entire pre-trained model.\
                             False: retrain the entire pre-trained model;\
                             This will override the 'transfer_cnn' and 'cnn' parameters;\
                             The appliance_name parameter will use to retrieve \
                             the entire pre-trained model of that appliance.")
    parser.add_argument("--transfer_cnn", type=str2bool,
                        default=False,
                        help="True: using a pre-trained CNN\
                              False: not using a pre-trained CNN.")
    parser.add_argument('--cnn',
                        type=str,
                        default='microwave',
                        help='The CNN trained by which appliance to load (pretrained model).')
    parser.add_argument('--gpus',
                        type=int,
                        default=-1,
                        help='Number of GPUs to use:\
                            n -- number of GPUs the system should use;\
                            -1 -- do not use any GPU.')
    parser.add_argument('--crop_dataset',
                        type=int,
                        default=10000,
                        help='for debugging porpose should be helpful to crop the training dataset size')
    parser.add_argument('--ram',
                        type=int,
                        default=5*10**5,
                        help='Maximum number of rows of csv dataset can handle without loading in chunks')
    return parser.parse_args()


In [6]:
args = get_arguments()
log('Arguments: ')
log(args)

2021-04-01 00:13:44,878 [INFO ]  Arguments: 
2021-04-01 00:13:44,879 [INFO ]  Namespace(appliance_name='microwave', batchsize=1000, cnn='microwave', crop_dataset=10000, datadir='/Users/shalinimustala/Documents/PPDS_coursework/DSCI560/Energy_disaggregation/Mypreprocessing_folder/trainingdata', dense_layers=1, fff='/Users/shalinimustala/Library/Jupyter/runtime/kernel-b738f530-330c-4d59-8d73-37a3fa0094cf.json', gpus=-1, n_epoch=5, pretrainedmodel_dir='./pretrained_model', ram=500000, save_dir='./models', save_model=-1, transfer_cnn=False, transfer_model=False)


In [7]:
# some constant parameters
CHUNK_SIZE = 5*10**6

# start the session for training a network
sess = tfc.InteractiveSession()

# the appliance to train on
appliance_name = args.appliance_name

# path for training data
training_path = args.datadir + '/' + 'REDD' + appliance_name + '_training_' + '.csv'
log('Training dataset: ' + training_path)

2021-04-01 00:13:44,924 [INFO ]  Training dataset: /Users/shalinimustala/Documents/PPDS_coursework/DSCI560/Energy_disaggregation/Mypreprocessing_folder/trainingdata/REDDmicrowave_training_.csv


In [8]:
# Looking for the validation set
for filename in os.listdir(args.datadir):
    if ("validation" in filename) and (appliance_name in filename):
        val_filename = filename
        log(val_filename)

# path for validation data
validation_path = args.datadir + '/' + val_filename
log('Validation dataset: ' + validation_path)

2021-04-01 00:13:44,935 [INFO ]  REDDmicrowave_validation_.csv
2021-04-01 00:13:44,937 [INFO ]  Validation dataset: /Users/shalinimustala/Documents/PPDS_coursework/DSCI560/Energy_disaggregation/Mypreprocessing_folder/trainingdata/REDDmicrowave_validation_.csv


In [9]:
# offset parameter from window length
offset = int(0.5*(params_appliance[args.appliance_name]['windowlength']-1.0))

# Defining object for training set loading and windowing provider (DataProvider.py)
tra_provider = ChunkDoubleSourceSlider2(filename=training_path,
                                        batchsize=args.batchsize,
                                        chunksize = CHUNK_SIZE,
                                        crop=args.crop_dataset,
                                        shuffle=True,
                                        offset=offset,
                                        header=0,
                                        ram_threshold=args.ram)

# Defining object for validation set loading and windowing provider (DataProvider.py)
val_provider = ChunkDoubleSourceSlider2(filename=validation_path,
                                        batchsize=args.batchsize,
                                        chunksize=CHUNK_SIZE,
                                        crop=args.crop_dataset,
                                        shuffle=False,
                                        offset=offset,
                                        header=0,
                                        ram_threshold=args.ram)


In [10]:
# TensorFlow placeholders
tf.compat.v1.disable_eager_execution()
x = tfc.placeholder(tf.float32,
                   shape=[None, params_appliance[args.appliance_name]['windowlength']],
                   name='x')

y_ = tfc.placeholder(tf.float32,
                    shape=[None, 1],
                    name='y_')


In [11]:
tf.__version__

'2.3.1'

In [12]:
#Don' run 
# -------------------------------- Keras Network - from model.py -----------------------------------------
inp = Input(tensor=x)
model, cnn_check_weights = get_model(args.appliance_name,
                                     inp,
                                     params_appliance[args.appliance_name]['windowlength'],
                                     transfer_dense=args.transfer_model,
                                     transfer_cnn=args.transfer_cnn,
                                     cnn=args.cnn,
                                     pretrainedmodel_dir=args.pretrainedmodel_dir)
y = model.outputs

2021-04-01 00:13:45,081 [INFO ]  Standard training...
2021-04-01 00:13:45,082 [INFO ]  ...creating a new model.


AttributeError: module 'numpy' has no attribute 'utils'

In [ ]:
#Don't run!
# cost function
cost = tf.reduce_mean(tf.reduce_mean(tfc.squared_difference(y, y_), 1))


In [ ]:

# model's weights to be trained
train_params = tfc.trainable_variables()
log("All network parameters: ")
log([v.name for v in train_params])
# if transfer learning is selected, just the dense layer will be trained
if not args.transfer_model and args.transfer_cnn:
    parameters = 10
else:
    parameters = 0
log("Trainable parameters:")
log([v.name for v in train_params[parameters:]])

In [ ]:
# Training hyper parameters
train_op = tfc.train.AdamOptimizer(learning_rate=0.001,
                                  beta1=0.9,
                                  beta2=0.999,
                                  epsilon=1e-08,
                                  use_locking=False).minimize(cost,
                                                              var_list=train_params[parameters:]
                                                              )

uninitialized_vars = []
for var in tf.all_variables():
    try:
        sess.run(var)
    except tf.errors.FailedPreconditionError:
        uninitialized_vars.append(var)
